<a href="https://colab.research.google.com/github/meghanabhange/Arctic-Monkeys-Lyrics-Generator/blob/master/Data_AM_LyricsGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's get the data first

To get all those amazing Arctic Monkeys lyrics I am using [lyricsgenius Python Package](https://github.com/johnwmillr/LyricsGenius)

First you need to get your API Keys, you can do that by registering your project here and generating a client access token

[Get your API key here](https://genius.com/api-clients)

![img](https://i.imgur.com/xmQDBgY.png)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import warnings

import requests

warnings.filterwarnings('ignore')

In [4]:
# !pip install lyricsgenius
# @title Genius API Details

import lyricsgenius as genius

API_KEY = "0kRvi0WFhTVqfsNzRlljlEfAKZpCfr5Xv4uEotCqUzI415HuB_eNdByfbehlk2Kv"  # @param {type:"string"}
ARTIST = "Arctic Monkeys"  # @param {type:"string"}

     |████████████████████████████████| 92kB 3.4MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [5]:
api = genius.Genius(API_KEY)
artist = api.search_artist(ARTIST)
print(artist.songs)

Searching for songs by Arctic Monkeys...

Song 1: "Do I Wanna Know?"
Song 2: "Arabella"
Song 3: "I Wanna Be Yours"
Song 4: "R U Mine?"
Song 5: "505"
Song 6: "Knee Socks"
Song 7: "Why’d You Only Call Me When You’re High?"
Song 8: "Fluorescent Adolescent"
Song 9: "No. 1 Party Anthem"
Song 10: "Four Out of Five"
Song 11: "Star Treatment"
Song 12: "Snap Out of It"
Song 13: "Tranquility Base Hotel & Casino"
Song 14: "One for the Road"
Song 15: "Suck It and See"
Song 16: "Cornerstone"
Song 17: "Fireside"
Song 18: "One Point Perspective"
Song 19: "Crying Lightning"
Song 20: "Mardy Bum"
Song 21: "The Ultracheese"
Song 22: "I Bet You Look Good on the Dancefloor"
Song 23: "Teddy Picker"
Song 24: "When the Sun Goes Down"
Song 25: "Stop the World I Wanna Get Off With You"
Song 26: "Do Me a Favour"
Song 27: "Batphone"
Song 28: "I Want It All"
Song 29: "Love Is a Laserquest"
Song 30: "Only Ones Who Know"
Song 31: "American Sports"
Song 32: "Brianstorm"
Song 33: "A Certain Romance"
Song 34: "She Look

## We can save all the lyrics as JSON

In [6]:
artist.save_lyrics()

Wrote `Lyrics_ArcticMonkeys.json`


Skipping other details like Album name, release date and much more to keep what's relevant to use. The Lyrics

In [0]:
import json

with open("Lyrics_ArcticMonkeys.json") as f:
    data = json.load(f)

In [8]:
import pandas as pd

df = pd.DataFrame(data["songs"])[["title", "lyrics"]]
df.dropna(inplace=True)
df.head()

,title,lyrics
0,Do I Wanna Know?,[Verse 1]\nHave you got colour in your cheeks?...
1,Arabella,[Verse 1]\nArabella's got some interstellar-ga...
2,I Wanna Be Yours,[Verse 1]\nI wanna be your vacuum cleaner\nBre...
3,R U Mine?,[Verse 1]\nI'm a puppet on a string\nTracy Isl...
4,505,[Chorus]\nI'm going back to 505\nIf it's a sev...


getting rid of some mess

In [9]:
df["lyrics"] = (
    df["lyrics"]
    .apply(lambda x: x.lower())
    .apply(lambda x: x.replace("\\", " "))
    .apply(lambda x: x.replace("\\n", " "))
    .apply(lambda x: x.replace("\n", " "))
    .apply(lambda x: x.replace("'", ""))
)
df.head()

,title,lyrics
0,Do I Wanna Know?,[verse 1] have you got colour in your cheeks? ...
1,Arabella,[verse 1] arabellas got some interstellar-gato...
2,I Wanna Be Yours,[verse 1] i wanna be your vacuum cleaner breat...
3,R U Mine?,[verse 1] im a puppet on a string tracy island...
4,505,[chorus] im going back to 505 if its a seven-h...


# Text Augumentation

In [10]:
!pip install nlpaug
!pip install transformers

     |████████████████████████████████| 102kB 2.6MB/s 
     |████████████████████████████████| 645kB 3.5MB/s 
     |████████████████████████████████| 3.8MB 17.7MB/s 
     |████████████████████████████████| 890kB 31.2MB/s 
     |████████████████████████████████| 1.1MB 42.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=7b7f7a3839247c1a6d076c71a1175870c80e2ec19c5ebe6aa8beacbf985004e8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
import os
import unittest
from os.path import dirname, join

import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.flow as naf
from nlpaug import *
from nlpaug.augmenter.word import ContextualWordEmbsAug
from nlpaug.util import Action

# Contextual WordEmbs Augumentation

In [12]:
# @title Test Contextual WordEmbs Augumentation
aug = naw.ContextualWordEmbsAug(model_path="bert-base-uncased", action="insert")

test_text = "have you got colour in your cheeks?"  # @param {type:"string"}
augmented_text = aug.augment(test_text)
print("Original:")
print(test_text)
print("Augmented Text:")
print(augmented_text)


Original:
have you got colour in your cheeks?
Augmented Text:
what have you got colour in your yellow cheeks ?


# Contextual WordEmbs Augumentation in a flow 

In [13]:
# @title Test Contextual WordEmbs Augumentation
aug = naf.Sequential(
    [
        ContextualWordEmbsAug(action=Action.INSERT),
        ContextualWordEmbsAug(action=Action.SUBSTITUTE),
        naw.RandomWordAug(),
    ]
)
test_text = "there is always somebody taller with more of a wit"  # @param {type:"string"}
aug.augment(test_text)

'there is also much taller and with a more rounded'

In [0]:
def splitter(n, s):
    pieces = s.split()
    return [str(" ".join(pieces[i : i + n])) for i in range(0, len(pieces), n)]

In [19]:
text = " ".join(df.lyrics)
len(text)

172477

In [0]:
# write a txt file for just the original dataset
text_file = open("drive/My Drive/ArcticMonkeys/am_corpus_original.txt", "w")
text_file.write(text)
text_file.close()

In [21]:
from progressbar import ProgressBar

pbar = ProgressBar()

augumented = []
for s in pbar(splitter(10, text)):
    try:
        augumented.append(aug.augment(s))
    except ValueError:
        pass

100% (3306 of 3306) |####################| Elapsed Time: 0:06:53 Time:  0:06:53


In [22]:
augumented_text = str(" ".join(augumented)).replace("#", "") + str(
    " ".join(augumented_fasttext)
).replace("#", "")
corpus = text + augumented_text
print(
    f"len text : {len(text)}, Length of Augmumnted = {len(str(' '.join(augumented)).replace('#', ''))}, Length of FasttextAUG = {len(str(' '.join(augumented_fasttext)).replace('#', ''))}"
)
print(f"Percentage of Orignal Text : {len(text)*100/len(corpus)}")
print(f"Percentage of Augumented Text : {len(augumented_text)*100/len(corpus)}")

# write a text file for the whole corpus
text_file = open("drive/My Drive/ArcticMonkeys/am_corpus.txt", "w")
text_file.write(corpus)
text_file.close()

len text : 172477, Length of Augmumnted = 155249, Length of FasttextAUG = 0
Percentage of Orignal Text : 52.62841520050286
Percentage of Augumented Text : 47.37158479949714
